In [9]:
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm
from MLGenerateData import MLGenerateData
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
import numpy as np 

In [5]:
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import numpy as np 
from State import State
import random

data = pd.read_csv("6_24months_US_car_data.csv")
# data = pd.read_csv("src/6_24months_mldata.csv")
# data = pd.read_csv("src/72_onlinemldata.csv")
# data = pd.read_csv("src/144_mldata.csv.csv")

target = data['profit']
predictors = data.drop(['profit', data.columns[0]], axis=1)

print(predictors)
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.33, random_state=42)
n_cols = predictors.shape[1]


# target = data.df['profit']
# predictors = data.df.drop(['profit'], axis=1)
# X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.33, random_state=42)

# regr = linear_model.LinearRegression()
# regr.fit(X_train, y_train)

# print('Intercept: \n', regr.intercept_)
# print('Coefficients: \n', regr.coef_)

# preds = regr.predict(X_test)
# print(MSE(y_test, preds))

X_train = sm.add_constant(X_train)
model = sm.OLS(y_train.astype(float), X_train.astype(float)).fit()

X_test = sm.add_constant(X_test)
preds = model.predict(X_test.astype(float))

df = pd.read_csv("./TOTALSA.csv")
mean = df['TOTALSA'].mean()
std = df['TOTALSA'].std()

tests = [[54, 63, 42, 47, 42, 49], np.round([42.65454086832623, 55.47066141610458, 45.23110339190882 , 69.56550258424139, 30.0, 50.14861820830117])]

def test(model, mean, std, tests):
    state = State()
    state.create_state([-1 ,0, 1, 1, 2, 2], mean=mean, std=std)
    curr_max = 0
    lst = []
    for x in range(1000000):
        # s_DC1, S_DC1, s_DC2, S_DC2, s_r1, S_r1 = [random.randint(round(mean * 2), round(mean * 4)), random.randint(round(mean * 5), round(mean * 8)), random.randint(round(mean * 2), round(mean * 4)), random.randint(round(mean * 5), round(mean * 8)), random.randint(round(mean * 2), round(mean * 4)), random.randint(round(mean * 5), round(mean * 8))]
        s_DC1, S_DC1, s_DC2, S_DC2, s_r1, S_r1 = [random.randint(round(mean * 2), round(mean * 10)), random.randint(round(mean * 2), round(mean * 10)), random.randint(round(mean * 2), round(mean * 10)), random.randint(round(mean * 2), round(mean * 10)), random.randint(round(mean * 2), round(mean * 10)), random.randint(round(mean * 2), round(mean * 10))]
        if s_DC1 >= S_DC1 or s_DC2 >= S_DC2 or s_r1 >= S_r1:
            continue

        params = [1, mean, std, s_DC1, S_DC1, s_DC2, S_DC2, s_r1, S_r1]
        preds = model.predict(params)
        if preds > curr_max:
            curr_max = preds
            lst = [s_DC1, S_DC1, s_DC2, S_DC2, s_r1, S_r1]
    for x in tests:
        params = [1, mean, std]
        params.extend(x)
        preds = model.predict(params)
        if preds > curr_max:
            curr_max = preds
            lst = x

    return [curr_max, lst]

# print('result', model.predict([1, mean, std, 0.0, 54, 63, 42, 47, 42, 49]))

print(test(model, mean, std, tests))

def vif_analysis(data):
    """VIF analysis on the data dataframe"""
    for i in range(len(data.columns)):
        v=vif(np.matrix(data),i)
        print("Variance inflation factor for {}: {}".format(data.columns[i],round(v,2)))

vif_analysis(X_train)





print(model.summary())
print("mse: " + str(MSE(y_test, preds)))
print("mae: " + str(MAE(y_test, preds)))
print("mape: " + str(MAPE(y_test, preds)))

           mean       std  s_DC1  S_DC1  s_DC2  S_DC2   s_r1   S_r1
0     13.289917  0.447491   28.0   92.0   46.0   96.0   31.0   36.0
1     13.289917  0.447491   92.0  128.0  109.0  113.0   53.0   70.0
2     13.289917  0.447491   53.0  126.0   57.0   76.0   52.0   54.0
3     13.289917  0.447491  110.0  127.0   27.0   49.0  107.0  132.0
4     13.289917  0.447491   79.0  108.0   55.0  117.0   51.0  133.0
...         ...       ...    ...    ...    ...    ...    ...    ...
2851  15.129236  2.265378   52.0  110.0   56.0  100.0   42.0  101.0
2852  15.129236  2.265378   69.0  117.0  111.0  133.0  118.0  134.0
2853  15.129236  2.265378   61.0  125.0   95.0   99.0   62.0   93.0
2854  15.129236  2.265378  104.0  133.0   42.0   95.0  105.0  108.0
2855  15.129236  2.265378   46.0   88.0   32.0  101.0  121.0  130.0

[2856 rows x 8 columns]


/Users/nicholas/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


[array([116462.92177014]), [37, 44, 30, 75, 31, 33]]
Variance inflation factor for const: 498.78
Variance inflation factor for mean: 1.29
Variance inflation factor for std: 1.21
Variance inflation factor for s_DC1: 1.35
Variance inflation factor for S_DC1: 1.39
Variance inflation factor for s_DC2: 1.35
Variance inflation factor for S_DC2: 1.37
Variance inflation factor for s_r1: 1.35
Variance inflation factor for S_r1: 1.36
                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.972
Model:                            OLS   Adj. R-squared:                  0.972
Method:                 Least Squares   F-statistic:                     8291.
Date:                Mon, 03 Jul 2023   Prob (F-statistic):               0.00
Time:                        09:46:06   Log-Likelihood:                -17841.
No. Observations:                1913   AIC:                         3.570e+04
Df Residuals:      

In [11]:
data = pd.read_csv("./6_24months_car_data.csv")
target = data['profit']
predictors = data.drop(['profit'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.33, random_state=42)
n_cols = predictors.shape[1]
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train.astype(float), X_train.astype(float)).fit()

X_test = sm.add_constant(X_test)
preds = model.predict(X_test.astype(float))


print(model.summary())
print("mse: " + str(MSE(y_test, preds)))
print("mae: " + str(MAE(y_test, preds)))
print("mape: " + str(MAPE(y_test, preds)))

/Users/nicholas/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     11.95
Date:                Mon, 12 Jun 2023   Prob (F-statistic):           1.09e-51
Time:                        12:37:21   Log-Likelihood:                -25035.
No. Observations:                1343   AIC:                         5.013e+04
Df Residuals:                    1311   BIC:                         5.030e+04
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.853e+06    9.9e+06      0.692      0.4

In [12]:
data = pd.read_csv("./144_US_car_data.csv")
target = data['profit']
predictors = data.drop(['profit'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.33, random_state=42)
n_cols = predictors.shape[1]
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train.astype(float), X_train.astype(float)).fit()

X_test = sm.add_constant(X_test)
preds = model.predict(X_test.astype(float))


print(model.summary())
print("mse: " + str(MSE(y_test, preds)))
print("mae: " + str(MAE(y_test, preds)))
print("mape: " + str(MAPE(y_test, preds)))

/Users/nicholas/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.902
Method:                 Least Squares   F-statistic:                     398.0
Date:                Mon, 12 Jun 2023   Prob (F-statistic):               0.00
Time:                        12:37:22   Log-Likelihood:                -33403.
No. Observations:                3164   AIC:                         6.695e+04
Df Residuals:                    3090   BIC:                         6.740e+04
Df Model:                          73                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.358e+04   1253.062     10.834      0.0

In [13]:
data = pd.read_csv("./144_car_data.csv")
target = data['profit']
predictors = data.drop(['profit'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.33, random_state=42)
n_cols = predictors.shape[1]
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train.astype(float), X_train.astype(float)).fit()

X_test = sm.add_constant(X_test)
preds = model.predict(X_test.astype(float))


print(model.summary())
print("mse: " + str(MSE(y_test, preds)))
print("mae: " + str(MAE(y_test, preds)))
print("mape: " + str(MAPE(y_test, preds)))

/Users/nicholas/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.739
Model:                            OLS   Adj. R-squared:                  0.724
Method:                 Least Squares   F-statistic:                     49.17
Date:                Mon, 12 Jun 2023   Prob (F-statistic):          3.37e-316
Time:                        12:37:22   Log-Likelihood:                -22973.
No. Observations:                1343   AIC:                         4.609e+04
Df Residuals:                    1269   BIC:                         4.648e+04
Df Model:                          73                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        8.76e+06   2.21e+06      3.955      0.0